In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
from data_preparation import Preprocessing
import pickle
# -import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

import os
from tensorflow.python.framework import ops
from sklearn.metrics.classification import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
import warnings

Instructions for updating:
non-resource variables are not supported in the long term


/home/mike/miniconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
# Set the environment 
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
ops.reset_default_graph()

# Give the path to the three necessary directory. 
checkpoints_dir = '/home/mike/Documents/keras_lstm/LSTM_Sentiment/checkpoints/1603011907' 
data_dir = '/home/mike/Documents/keras_lstm/LSTM_Sentiment/data/' 
stopwords_file = '/home/mike/Documents/keras_lstm/LSTM_Sentiment/data/stopwords.txt' 

sequence_len = None # Maximum sequence length
n_samples= None # Set n_samples=None to use the whole dataset
test_size = 0.2
batch_size = 100 # Batch size
random_state = 0 # Random state used for data splitting. Default is 0

if checkpoints_dir is None:
    raise ValueError('Please, a valid checkpoints directory is required (--checkpoints_dir <file name>)')

In [4]:
# Load the data
data_lstm = Preprocessing(data_dir=data_dir,
                          stopwords_file=stopwords_file,
                          sequence_len=sequence_len,
                          n_samples=n_samples,
                          test_size=test_size,
                          val_samples=batch_size,
                          random_state=random_state,
                          ensure_preprocessed=True)

Loading preprocessed files ...


In [5]:
# Import the graph and evaluate the model with using test data
original_text, x_test, y_test, test_seq_len = data_lstm.get_test_data(original_text=True)
graph = tf.Graph()

with graph.as_default():
    sess = tf.Session()

    # Import the graph and restore its weights
    print('Restoring graph ...')
    saver = tf.train.import_meta_graph("{}/model.ckpt.meta".format(checkpoints_dir))
    saver.restore(sess, ("{}/model.ckpt".format(checkpoints_dir)))

    # Recover input/output tensors
    input = graph.get_operation_by_name('input').outputs[0]
    target = graph.get_operation_by_name('target').outputs[0]
    seq_len = graph.get_operation_by_name('lengths').outputs[0]
    dropout_keep_prob = graph.get_operation_by_name('dropout_keep_prob').outputs[0]
    predict = graph.get_operation_by_name('final_layer/softmax/predictions').outputs[0]
    accuracy = graph.get_operation_by_name('accuracy/accuracy').outputs[0]

    # Perform the prediction
    pred, acc = sess.run([predict, accuracy], feed_dict={input: x_test, target: y_test, seq_len: test_seq_len, dropout_keep_prob: 1})
    print("Evaluation done.")

Restoring graph ...
INFO:tensorflow:Restoring parameters from /home/mike/Documents/keras_lstm/LSTM_Sentiment/checkpoints/1603011907/model.ckpt
Evaluation done.


In [6]:
# Show the scalars and the graph
%tensorboard --logdir='/home/mike/Documents/keras_lstm/LSTM_Sentiment/logs'

In [7]:
# Print the results
print('\nAccuracy: {0:.4f}\n'.format(acc))
for i in range(100):
    print('Sample: {0}'.format(original_text[i]))
    print('Predicted sentiment: [{0:.4f}, {1:.4f}]'.format(pred[i, 0], pred[i, 1]))
    print('Real sentiment: {0}\n'.format(y_test[i]))


Accuracy: 0.9858

Sample: [3143 1
 'Brokeback Mountain is THE most amazing / beautiful / romantic / heartbraking movie i have ever or will ever see in my life......']
Predicted sentiment: [0.0000, 1.0000]
Real sentiment: [0. 1.]

Sample: [2178 1
 'I love Harry Potter ( the books are much better than the movies ).']
Predicted sentiment: [0.0000, 1.0000]
Real sentiment: [0. 1.]

Sample: [5078 0 'Mission Impossible 3 et al. can be said to be boring!']
Predicted sentiment: [1.0000, 0.0000]
Real sentiment: [1. 0.]

Sample: [4165 0
 'In other news, The Da Vinci code sucks and Symantec is suing Microsoft...']
Predicted sentiment: [1.0000, 0.0000]
Real sentiment: [1. 0.]

Sample: [150 1
 'lynn and jon at the theaters for da vinci code-other than that, EVERYONE AT THE LUAU, it was awesome..']
Predicted sentiment: [0.0000, 1.0000]
Real sentiment: [0. 1.]

Sample: [4195 0
 'Combining the opinion / review from Gary and Gin Zen, The Da Vinci Code sucks.']
Predicted sentiment: [1.0000, 0.0000]
Real